In [ ]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

# Build LSTM model used to do predictions
model = Sequential()
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=50, activation="relu"))
model.add(Dense(units=1))

model.compile(optimizer="adam", loss="mean_squared_error")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from rich.jupyter import print


# Number of past rounds to sequence for the model to consider when predicting
# the growth of a player for round.
n_steps: int = 10

scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))


def prepare_data(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    """
    Prepare the data for training the model. This includes normalizing the data
    and combining the data into a X and y array.
    """

    # NaN values is often because no stat was recorded for the given player for
    # that game. That means that the player was not on the pitch and thus 0 is
    # the proper value to use.
    df.fillna(0, inplace=True)

    # Normalize the data to gain better results. Using separate scaler to allow
    # for independently inverting again.
    df[df.columns.difference(["growth"])] = scaler_x.fit_transform(
        df[df.columns.difference(["growth"])]
    )
    df["growth"] = scaler_y.fit_transform(df[["growth"]])

    # Combine number of time_steps into a single array and set the target to the
    # growth of the following round. E.g. combine values from round 1-10 and set
    # the target to the growth of round 11.
    time_steps = n_steps
    features = []
    target = []
    for _, group in df.groupby("id"):
        # Drop columns that will overfit the data. The id and round have no real
        # influence on the players performance. We only keep the id to be able
        # to group the data by player.
        group.drop(columns=["id", "round"], inplace=True)

        for i in range(time_steps, len(group)):
            features.append(group.iloc[i - time_steps : i].values)
            target.append(group.iloc[i]["growth"])

    return np.array(features), np.array(target)


In [ ]:
from sklearn.model_selection import train_test_split


def train(df: pd.DataFrame, epochs: int = 10, batch_size: int = 32) -> None:
    """
    Train the LSTM model using the given data frame. Returns the model for use
    to predict the growth of a player.
    """
    X, y = prepare_data(df)

    # Split the data into training and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Train the model
    model.fit(
        X_train,
        y_train,
        # Number of iterations over the entire dataset
        epochs=epochs,
        # Number of samples per gradient update
        batch_size=batch_size,
        # Use 20% of the data for validation
        validation_split=0.2,
    )

    loss = model.evaluate(X_test, y_test)
    print(f"\nModel evaluation: {loss=!r}")


In [ ]:
from holdet.data import sofascore
from holdet.campaign import Game, BaseCandidate, Round
from functools import cache


class CandidateLTSM(BaseCandidate):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def features(self, stat: sofascore.Statistics) -> dict[str, float | int]:
        """
        Return a dictionary with all the features that can be used to train a
        model.
        """

        # TODO: Add data from betting sites
        return {
            "substitute": int(stat.substitute),
            "assists": stat.assists,
            "expectedAssists": stat.expectedAssists,
            "expectedGoals": stat.expectedGoals,
            "goals": stat.goals,
            "goalsPrevented": stat.goalsPrevented,
            "minutesPlayed": stat.minutesPlayed,
            "onTargetScoringAttempt": stat.onTargetScoringAttempt,
            "savedShotsFromInsideTheBox": stat.savedShotsFromInsideTheBox,
            "saves": stat.saves,
            "team_goals": stat.team_goals,
            "team_goals_conceded": stat.team_goals_conceded,
            "win": int(stat.win),
            "loss": int(stat.loss),
            "draw": int(stat.draw),
            "clean_sheet": int(stat.clean_sheet),
            "decisive_goal_for_draw": int(stat.decisive_goal_for_draw),
            "decisive_goal_for_win": int(stat.decisive_goal_for_win),
        }

    def aggregate_stats(self, round: Round) -> dict:
        """
        Aggregate all the stats from a round into one. This is needed for rounds
        with multiple games in them. The features are summed together.

        Its not ideal for stuff like opponent or side which will simply be added
        together, but thats we can do for now.
        """
        round_stats: dict[str, int | float] = {}

        for stat in round.stats:
            for key, value in self.features(stat).items():
                round_stats[key] = round_stats.get(key, 0) + value
        return round_stats

    def round_features(self, round: Round) -> dict:
        """
        Return a dictionary with all the features that can be used to train a
        model.
        """

        features = {
            "id": self.id,
            "round": round.number,
            "position": round.position.value,
            "team": self.team_id,
            "games": len(round.schedules),
            "growth": round.growth,
        }

        # Assume that any player have at max 2 scheduled games in a round.
        # Append the opponent and side for each game or None to the features.
        team = self.avatar.player.team
        for idx in range(2):
            if idx < len(round.schedules):
                schedule = round.schedules[idx]
                opponent_id = schedule.opponent(team).id
                side = schedule.side(team).value
            else:
                opponent_id = None
                side = None
            features[f"opponent_{idx}"] = opponent_id
            features[f"side_{idx}"] = side

        # Aggregate all the features from the round
        stats = self.aggregate_stats(round)
        for key, value in stats.items():
            features[key] = value

        return features

    def generate_dataframe(self) -> pd.DataFrame:
        """
        Generate a dataframe for the candidate with features for every round
        """
        data = []

        for round in self.rounds:
            data.append(self.round_features(round))

        return pd.DataFrame(data)

    def next_dataframe(self, n: int) -> pd.DataFrame:
        """
        Generate a dataframe for the candidate with features for the n next
        rounds
        """
        data = []

        for round in self.next_rounds(n):
            data.append(self.round_features(round))

        return pd.DataFrame(data)

    def xGrowth(self, n: int = 5) -> float:
        """
        Predict the combined growth from n next rounds
        """
        data = self.generate_dataframe()
        n_data = self.next_dataframe(n)
        df = pd.concat([data, n_data])
        X, _ = prepare_data(df)
        y = model.predict(X)

        # Sum the growth from n last rounds in prediction, this is the same
        # number of next rounds included in the dataframe.
        inverse_y = scaler_y.inverse_transform(y)
        x = inverse_y[:-n].sum()
        return x

    @property
    def xValue(self) -> float:
        try:
            xGrowth = self.xGrowth(n=5)
            if self.captain:
                return self.value + xGrowth * 2
            return self.value + xGrowth
        except Exception as err:
            print(f"{self.name}: {err}")
            return 0


game = Game.new(CandidateLTSM)


In [ ]:
# Generate a dataframe for each candidate, concat into one and use it to train
# the model.
data = []
for candidate in game.candidates:
    candidate: CandidateLTSM
    data.append(candidate.generate_dataframe())
df = pd.concat(data)

train(df, epochs=10, batch_size=32)


In [ ]:
from holdet.solver import lp

budget = 50 * 1000000  # 50 million

# Manual data cleanup. Mostly injured players.
for c in game.candidates:
    if c.name == "Granit Xhaka":
        game.candidates.remove(c)
    if c.name == "Tom Cairney":
        game.candidates.remove(c)
    if c.name == "Bryan Mbeumo":
        game.candidates.remove(c)
    if c.name == "Ivan Toney":
        game.candidates.remove(c)
    if c.name == "Rodrigo Bentancur":
        game.candidates.remove(c)
    if c.value == 0:
        game.candidates.remove(c)


# Calculate the best starting 11 within the budget using linear programming
solution = lp.find_optimal_team(game.candidates, budget)


In [ ]:
from holdet.formation import Formation

print(Formation(solution))
